# Train a model locally
*Base code from the [Keras Applications](https://keras.io/api/applications/) page.*

In [49]:
# import libraries from keras
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [2]:
# # create the base pre-trained model
# # note that if this does not run, downgrade h5py
# # pip install "h5py==2.10.0" --force-reinstall
# base_model = applications.InceptionV3(weights = 'imagenet',
#                                       include_top=False,
#                                       input_shape = (558,558,3))

In [3]:
# # add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(2, activation='softmax')(x)

In [4]:
# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#     layer.trainable = False

In [6]:
# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

### Hand-built model
*Following [this](https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/) tutorial.*

In [22]:
def define_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(558, 558, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
model = define_model()

In [24]:
datagen = image.ImageDataGenerator()

In [41]:
# prepare iterators
train_it = datagen.flow_from_directory('../data/images-model/train/',
    class_mode='binary', batch_size=64, target_size=(558, 558), shuffle = False)
test_it = datagen.flow_from_directory('../data/images-model/test/',
    class_mode='binary', batch_size=64, target_size=(558, 558), shuffle = False)

Found 1807 images belonging to 2 classes.
Found 603 images belonging to 2 classes.


In [42]:
# fit model
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=test_it, validation_steps=len(test_it), epochs=2, verbose=0)

In [46]:
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))

> 82.919


In [50]:
#Confusion Matrix and Classification Report
Y_pred = model.predict_generator(test_it, 603 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_it.classes, y_pred))
print('Classification Report')
target_names = ['no-kithara', 'kithara']
print(classification_report(test_it.classes, y_pred, target_names=target_names))

Confusion Matrix


ValueError: Found input variables with inconsistent numbers of samples: [603, 1179]

In [ ]:
test_data = []
test_labels = []
batch_index = 0

while batch_index <= test_generator.batch_index:
    data = next(test_generator)
    test_data.append(data[0])
    test_labels.append(data[1])
    batch_index = batch_index + 1

test_data_array = np.asarray(test_data)
test_labels_array = np.asarray(test_labels)

In [ ]:
# train the model on the new data for a few epochs
model.fit()

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')